## Import Statements

In [1]:
#!pip install geehydro
import ee
import geemap
import folium
import geehydro

In [2]:
ee.Initialize()

## Initialize Map

In [3]:
Map = geemap.Map(center=(40, -100), zoom=4) #use "Map" to be consistent with Javascript API


In [4]:
landsatData = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2021-05-01', '2021-06-01'); #Landsat 8 data from May 2021
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover') #Lan cover data 
states = ee.FeatureCollection("TIGER/2018/States") #add states 

In [5]:
landsat_vis = {
    'min': 6000,
    'max': 13000,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

In [6]:
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(landsatData, landsat_vis, 'Landsat 8')
Map.addLayer(states, {}, "US States")

## Get Data, Plot Desired Location

In [7]:
#St Louis Coordinates
stllon = -90.1994
stllat = 38.627

#Pennington coordinates
plon = -74.7888
plat = 40.3249

#Set point for desired coordinates
point = ee.Geometry.Point(plon, plat)

#iniialize image for area of analysis
image = ee.Image(landsatData.filterBounds(point).sort('CLOUD_COVER').first());



## Compute and Plot NDVI

In [8]:
#define a method that computes NDVI using the build in normalizedDifference() method utilizing bands 4 and 5 for Landsat 8
def getNDVI(img):
    return img.normalizedDifference(['SR_B5','SR_B4']) 

#compute NDVI!
ndvi = getNDVI(image)

#define a color palette
ndviParams = {'palette': ["#d73027", "#f46d43","#fdae61","fee08b","d9efb8","a6d96a","66bd63","1a9850"]} 
Map.addLayer(ndvi, ndviParams, 'NDVIMAP')
Map.centerObject(ndvi, 7)

In [9]:
Map.setControlVisibility(layerControl= True, fullscreenControl = True, latLngPopup = True)
Map

Map(center=[40.331277783812, -74.88151495388823], controls=(WidgetControl(options=['position', 'transparent_bg…

## sources

1. https://www.bing.com/videos/search?q=ndvi+calculation+google+earth+engine&&view=detail&mid=4684F2F5965E9701F7354684F2F5965E9701F735&&FORM=VRDGAR&ru=%2Fvideos%2Fsearch%3Fq%3Dndvi%2Bcalculation%2Bgoogle%2Bearth%2Bengine%26FORM%3DHDRSC4

2. https://geemap.org/get-started/#plotting-backends